In [ ]:
# Gerekli kütüphaneleri yükleyelim
!pip install fuzzywuzzy python-levenshtein
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import numpy as np

# Tüm CSV'leri yükleyelim
try:
    df_watsons = pd.read_csv("watsons_analize_hazir_veri.csv")
    # products.csv dosyasında genellikle Barkod (EAN) yerine sadece ID olur.
    # Bu Rossmann'ın kendi verisi olduğu için temel karşılaştırma noktasıdır.
    df_rossmann = pd.read_csv("products.csv", encoding='utf-8')

    print(f"✅ Watsons Verisi: {len(df_watsons)} ürün")
    print(f"✅ Rossmann Verisi: {len(df_rossmann)} ürün")

except FileNotFoundError:
    print("❌ HATA: Gerekli CSV dosyaları (watsons_analize_hazir_veri.csv veya products.csv) bulunamadı.")
    exit()

# --- 2. VERİ TEMİZLEME VE HAZIRLIK ---

# A) Watsons Verisini Hazırlama (Kolon İsimleri Zaten Temiz)
df_watsons = df_watsons.rename(columns={'normal_fiyat': 'W_Fiyat'})
df_watsons['Marka_Urun'] = df_watsons['marka'] + " " + df_watsons['urun_adi']
df_watsons['kategori'] = df_watsons['kategori'].str.lower()


# B) Rossmann Verisini Hazırlama (Kolonları Eşleştirme)
# products.csv yapısına göre (sizin dosyanızın içeriğine göre varsayılmıştır)
# Rossmann'ın normal fiyatının 3. kolonda ve Marka bilgisinin 7. kolonda olduğunu varsayıyoruz.
# Eğer bu sütun indeksleri farklıysa, kodun bu kısmını ayarlamanız gerekir.

# Sadece gerekli kolonları seç ve yeniden adlandır
# 3, 7, 10 -> Rossmann fiyat, marka, ürün adı kolonları
df_rossmann_clean = df_rossmann.iloc[:, [1, 2, 6, 9]]
df_rossmann_clean.columns = ['R_Urun_Adi_Ham', 'R_Fiyat_Ham', 'R_Marka', 'R_Kategori_Ham']

# Marka ve Ürün Adı temizliği
df_rossmann_clean['R_Kategori'] = df_rossmann_clean['R_Kategori_Ham'].apply(lambda x: str(x).split(' > ')[0].lower() if isinstance(x, str) else None)
df_rossmann_clean['R_Urun_Adi'] = df_rossmann_clean['R_Urun_Adi_Ham'].apply(lambda x: str(x).strip())
df_rossmann_clean['R_Fiyat'] = pd.to_numeric(df_rossmann_clean['R_Fiyat_Ham'], errors='coerce')

df_rossmann_clean = df_rossmann_clean.dropna(subset=['R_Marka', 'R_Urun_Adi', 'R_Fiyat', 'R_Kategori'])
df_rossmann_clean['R_Marka'] = df_rossmann_clean['R_Marka'].astype(str) # Marka sütununu stringe çevir
df_rossmann_clean['R_Urun_Adi'] = df_rossmann_clean['R_Urun_Adi'].astype(str) # Ürün Adı sütununu stringe çevir
df_rossmann_clean['Marka_Urun'] = df_rossmann_clean['R_Marka'] + " " + df_rossmann_clean['R_Urun_Adi']
df_rossmann_clean = df_rossmann_clean.reset_index(drop=True)

print("\n--- 3. ÜRÜN EŞLEŞTİRME (FUZZY MATCHING) BAŞLIYOR ---")
print("Bu işlem CPU yoğun olduğu için biraz sürebilir...")

# --- 3. FUZZY MATCHING İŞLEMİ ---

def eslestir(row, choices, scorer, threshold=80):
    """Her Watsons ürünü için Rossmann ürün listesindeki en benzer ürünü bulur."""

    # 1. Aynı Kategori ve Marka içinden arama yap
    kategori = str(row['kategori'])
    marka = str(row['marka'])
    query = row['Marka_Urun']

    # Rossmann'dan filtrelenmiş eşleşme listesini al
    filtered_choices = choices[
        (choices['R_Kategori'].str.contains(kategori, na=False)) &
        (choices['R_Marka'].str.contains(marka, na=False))
    ]

    if filtered_choices.empty:
        return np.nan, 0, np.nan

    # Bulanık eşleştirme yap
    results = process.extractOne(
        query,
        filtered_choices['Marka_Urun'],
        scorer=scorer,
        score_cutoff=threshold
    )

    if results:
        match_text, score, index = results
        # Eşleşen satırı Rossmann veri setinden çek
        matched_row = filtered_choices.loc[index]
        return matched_row['R_Urun_Adi'], score, matched_row['R_Fiyat']
    else:
        return np.nan, 0, np.nan

# Watsons veri setine eşleştirme sonuçlarını ekleyelim (Sadece ilk 100 ürünü test edelim)
df_watsons_sample = df_watsons.head(200).copy() # Hız için ilk 200 ürünü test et

df_watsons_sample[['R_Urun_Adi_Match', 'Score', 'R_Fiyat_Match']] = df_watsons_sample.apply(
    lambda row: pd.Series(eslestir(
        row,
        df_rossmann_clean,
        scorer=fuzz.token_sort_ratio # Token Sort Ratio en güvenilir metottur
    )),
    axis=1
)

# --- 4. ANALİZ VE RAPORLAMA ---

# Eşleşenleri al
df_matched = df_watsons_sample.dropna(subset=['R_Fiyat_Match']).copy()

# Fiyat Endeksi Hesaplama: (Rakip Fiyat / Bizim Fiyat) * 100
# Endeks > 100 ise Rakip Pahalı, Endeks < 100 ise Rakip Ucuz
df_matched['Fiyat_Endeksi'] = (df_matched['W_Fiyat'] / df_matched['R_Fiyat_Match']) * 100
df_matched['Fiyat_Farki_TL'] = df_matched['W_Fiyat'] - df_matched['R_Fiyat_Match']


print("\n--- 5. FİNAL ANALİZ RAPORU (Watsons vs. Rossmann) ---")

# A) Genel Endeks
genel_endeks = df_matched['Fiyat_Endeksi'].mean()
print(f"🌍 Eşleşen Ürün Sayısı: {len(df_matched)}")
print(f"🌍 GENEL FİYAT ENDEKSİ: {genel_endeks:.2f}")

if genel_endeks > 100:
    print(f"💡 Rossmann (Biz) Watsons'a göre %{(genel_endeks - 100):.2f} daha UCUZ konumlanmış durumda.")
else:
    print(f"💡 Rossmann (Biz) Watsons'a göre %{(100 - genel_endeks):.2f} daha PAHALI konumlanmış durumda.")

# B) Kategori Bazlı Endeks
kategori_endeksi = df_matched.groupby('kategori')['Fiyat_Endeksi'].mean().sort_values(ascending=False).reset_index()

print("\n📊 KATEGORİ BAZLI FİYAT ENDEKSİ (Rossmann = 100):")
print(kategori_endeksi.round(2))

# C) Fiyat Farkı En Yüksek 5 Ürün (Fırsatlar)
en_pahali_fark = df_matched.sort_values(by='Fiyat_Farki_TL', ascending=False).head(5)
en_ucuz_fark = df_matched.sort_values(by='Fiyat_Farki_TL', ascending=True).head(5)

print("\n📈 RAKİBİN EN PAHALI OLDUĞU ÜRÜNLER (Rossmann'ın Fiyat Artırma Fırsatı):")
display(en_pahali_fark[['marka', 'urun_adi', 'W_Fiyat', 'R_Fiyat_Match', 'Fiyat_Farki_TL']])

print("\n📉 RAKİBİN EN UCUZ OLDUĞU ÜRÜNLER (Rossmann'ın Fiyat Düşürme İhtiyacı):")
display(en_ucuz_fark[['marka', 'urun_adi', 'W_Fiyat', 'R_Fiyat_Match', 'Fiyat_Farki_TL']])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Gerekli kütüphaneleri yükleyelim
!pip install scikit-learn
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display

# --- 1. VERİ YÜKLEME ---
try:
    df_watsons = pd.read_csv("watsons_analize_hazir_veri.csv")
    df_rossmann = pd.read_csv("products.csv", encoding='utf-8')

except FileNotFoundError:
    print("HATA: Gerekli CSV dosyaları bulunamadı.")
    exit()

# --- 2. ETL (TEMİZLİK VE VEKTÖRLEŞTİRMEYE HAZIRLIK) ---

print("🧹 ETL: Veri temizleniyor ve vektörleştirmeye hazırlanıyor...")

# Ortak temizleme fonksiyonu: Sayıları, birimleri ve özel karakterleri kaldırır
def metin_temizleyici(text):
    if not isinstance(text, str): return ""
    text = text.lower().strip()
    # Sayısal değerleri ve birimleri kaldır (50ml, 140adet vb. eşleştirmeyi bozar)
    text = re.sub(r'\s\d+[\s\w]+', ' ', text) # Sayı ve birimler
    text = re.sub(r'[^a-z0-9\s]', '', text) # Özel karakterler
    return text.strip()

# A) Watsons (Rakip) Verisi Temizliği
df_watsons = df_watsons.rename(columns={'normal_fiyat': 'W_Fiyat', 'marka': 'Ortak_Marka'})
df_watsons['W_Fiyat'] = pd.to_numeric(df_watsons['W_Fiyat'], errors='coerce')
df_watsons['Ortak_Marka'] = df_watsons['Ortak_Marka'].apply(metin_temizleyici)
df_watsons['urun_adi_vektor'] = df_watsons['urun_adi'].apply(metin_temizleyici)


# B) Rossmann (Bizim) Verisi Temizliği
df_rossmann_clean = df_rossmann.iloc[:, [1, 2, 7, 11]].copy()
df_rossmann_clean.columns = ['R_Urun_Adi_Ham', 'R_Fiyat_Ham', 'R_Marka', 'R_Kategori_Ham']

df_rossmann_clean['R_Fiyat'] = pd.to_numeric(df_rossmann_clean['R_Fiyat_Ham'], errors='coerce')
df_rossmann_clean['Ortak_Marka'] = df_rossmann_clean['R_Marka'].astype(str).apply(metin_temizleyici)
df_rossmann_clean['urun_adi_vektor'] = df_rossmann_clean['R_Urun_Adi_Ham'].astype(str).apply(metin_temizleyici)


# Ortak Marka ve Vektör Adı boş olanları çıkar
df_watsons = df_watsons.dropna(subset=['W_Fiyat', 'Ortak_Marka', 'urun_adi_vektor'])
df_rossmann_clean = df_rossmann_clean.dropna(subset=['R_Fiyat', 'Ortak_Marka', 'urun_adi_vektor'])


# --- 3. VEKTÖRLEŞTİRME (TF-IDF) ---

# Tüm temizlenmiş ürün adlarını birleştir
corpus = pd.concat([df_watsons['urun_adi_vektor'], df_rossmann_clean['urun_adi_vektor']])

# Basic Turkish stop words (for demonstration, a more comprehensive list might be needed)
turkish_stop_words = [
    'bir', 'bu', 'o', 'şu', 'işte', 'her', 'hiç', 'çok', 'en', 'daha', 'ki', 'da', 'de',
    've', 'ile', 'ama', 'fakat', 'ancak', 'lakin', 'yani', 'çünkü', 'eğer', 'ise', 'hem',
    'ya', 'yada', 'veyahut', 'oysa', 'halbu ki', 'halbuki', 'nitekim', 'zira', 'çünkü',
    'eğer ki', 'meğer', 'meğer ki', 'sanki', 'bile', 'tabi', 'bazen', 'hatta', 'üstelik'
]

# TF-IDF ile kelime önemini vektörlere dönüştür
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words=turkish_stop_words) # 1 ve 2 kelimelik grupları kullan
vectorizer.fit(corpus)

# Vektörleri oluştur
W_vectors = vectorizer.transform(df_watsons['urun_adi_vektor'])
R_vectors = vectorizer.transform(df_rossmann_clean['urun_adi_vektor'])

print(f"✅ Vektörleştirme bitti. Watsons: {W_vectors.shape[0]}x{W_vectors.shape[1]}")


# --- 4. EŞLEŞTİRME (KOSİNÜS BENZERLİĞİ) ---

def kosinus_eslestir(W_vectors, R_vectors, df_rossmann, threshold=0.5):
    """
    Kosinüs Benzerliği matrisini hesaplar ve her Watsons ürünü için en yakın Rossmann ürününü bulur.
    """
    print("\n⏳ Kosinüs Benzerliği Matrisi hesaplanıyor...")

    # Kosinüs Benzerliği Matrisi (Watsons vs Rossmann)
    similarity_matrix = cosine_similarity(W_vectors, R_vectors)

    match_results = []

    for i, w_row in df_watsons.iterrows():
        # Benzerlik skorları (i. Watsons ürünü için)
        scores = similarity_matrix[i]

        # En yüksek skoru bul
        best_match_index = np.argmax(scores)
        best_score = scores[best_match_index]

        # Eğer skor eşiğin üzerindeyse VE Markalar Eşleşiyorsa al
        # Temporarily remove strict brand and category checks to find any matches
        if best_score > threshold:

            # Watsons ve Rossmann kategorilerinin ilk harfleri de eşleşiyorsa (örneğin 'm'akya ile 'm'akyaj)
            # if w_row['kategori'][0] == df_rossmann.iloc[best_match_index]['R_Kategori_Ham'].lower()[0]:

                match_results.append({
                    'W_Urun_Adi': w_row['urun_adi'],
                    'W_Fiyat': w_row['W_Fiyat'],
                    'R_Urun_Adi': df_rossmann.iloc[best_match_index]['R_Urun_Adi_Ham'],
                    'R_Fiyat': df_rossmann.iloc[best_match_index]['R_Fiyat'],
                    'Score': best_score
                })

    return pd.DataFrame(match_results)


df_matched = kosinus_eslestir(W_vectors, R_vectors, df_rossmann_clean, threshold=0.05)

print(f"\n--- 5. FİNAL ANALİZ RAPORU (Kosinüs Benzerliği) ---")
print(f"🌍 Eşleşen Ürün Sayısı (Score > 0.05): {len(df_matched)}")

if df_matched.empty:
    print("⚠️ Yüksek eşik (0.05) sebebiyle eşleşme bulunamadı. Lütfen eşiği tekrar düşürmeyi deneyin.")
    exit()


# --- 6. ENDEKS HESAPLAMA ---
df_matched['Fiyat_Endeksi'] = (df_matched['W_Fiyat'] / df_matched['R_Fiyat']) * 100
df_matched['Fiyat_Farki_TL'] = df_matched['W_Fiyat'] - df_matched['R_Fiyat']

genel_endeks = df_matched['Fiyat_Endeksi'].mean()
print(f"🌍 GENEL FİYAT ENDEKSİ: {genel_endeks:.2f}")

# En büyük farkı göster
en_pahali_fark = df_matched.sort_values(by='Fiyat_Farki_TL', ascending=False).head(5)
print("\n📈 RAKİBİN EN PAHALI OLDUĞU ÜRÜNLER (Fırsat Alanları):")
display(en_pahali_fark[['W_Urun_Adi', 'W_Fiyat', 'R_Fiyat', 'Fiyat_Farki_TL', 'Score']].round(2))